## <font color='darkblue'>Preface</font>
This part will cover:
* The null reference, or “the billion-dollar mistake”
* Alternatives to null references
* Developing an Option data type for optional data
* Applying functions to optional values
* Composing optional values
* Option use cases

<font size='3ptx'><b>Representing optional data in computer programs has always been a problem</b></font>. The concept of optional data is very simple in everyday life. Representing the absence of something when this something is contained in a container is easy—whatever it is, it can be represented by an empty container. An absence of apples can be represented by an empty apple basket. The absence of gasoline in a car can be visualized as an empty gas tank. Representing the absence of data in computer programs is more difficult. Most data is represented as a reference pointing to it, so the most obvious way to represent the absence of data is to use a pointer to nothing. This is what a null pointer is.

In Python, a variable is a pointer to a value. Variables may be created [**None**](https://docs.python.org/2/library/constants.html#None) , and they may then be changed to point to values. They can even be changed again to point to null if data is removed. To handle optional data, in this chapter, you’ll develop the class called <font color='blue'><b>Option</b></font>. <b>The goal of this chapter is to learn how this kind of structure works in handling optional data</b>.

## <font color='darkblue'>Problems with the null pointer</font>
One of the most frequent bugs in imperative programs is the [**AttributeError**](https://docs.python.org/3/library/exceptions.html#AttributeError). <b>This error is raised when an identifier is dereferenced and found to be pointing to nothing. In other words, some data is expected but is found missing</b>. Such an identifier is said to be pointing to null. The null reference was invented in 1965 by Tony Hoare while he was designing the [**ALGOL**](https://en.wikipedia.org/wiki/ALGOL) object-oriented language. Here’s what he said 44 years later:

### <font color='darkgreen'>A normal imperative function code snippet</font>
Below is a normal Python code snippet which we will use learned FP concepts here to rewrite it:

In [106]:
class User:
    def __init__(self, first_name, second_name, email):
        self.first_name = first_name
        self.second_name = second_name
        self.email = email
        
    def __str__(self):
        email_as_string = self.email or "No data"
        return "{}/{} ({})".format(self.first_name, self.second_name, email_as_string)

    
def get_email_from_users(users: list[User|None], default_email: str) -> str:
    email_list = []
    for user in users:
        if user is not None and user.email is not None:
            email_list.append(user.email)
        else:
            email_list.append(default_email)
            
    return email_list

In [107]:
users = [
    User("Mickey", "Mouse", "mickey@disney.com"),
    None,
    User("Minnie", "Mouse", None),
]

for u in users:
    print(u)

Mickey/Mouse (mickey@disney.com)
None
Minnie/Mouse (No data)


In [108]:
print(get_email_from_users(users, 'no email'))

['mickey@disney.com', 'no email', 'no email']


### <font color='darkgreen'>The Option data type</font>
The <b><font color='blue'>Option</font></b> data type you’ll create in this chapter will be very similar to the [**List**](https://github.com/johnklee/fpu/blob/master/fpu/flist.py) data type. Using an <b><font color='blue'>Option</font></b> type for optional data allows you to compose functions even when the data is absent (<font color='brown'>see figure 6.1</font>). It will be implemented as an abstract class, <b><font color='blue'>Option</font></b>, <b>containing two private subclasses representing the presence and the absence of data</b>. The subclass representing the absence of data will be called <b><font color='blue'>N</font></b> (<font color='brown'>Stands for None</font>), and the subclass representing the presence of data will be called <b><font color='blue'>Some</font></b>. A <b><font color='blue'>Some</font></b> will contain the corresponding data value.

![null](https://3.bp.blogspot.com/-55VlPQ6TLEk/WkuDRgPMWII/AAAAAAAAXII/Yj_WnnSeS4UvBMzDflzhwJoz4E8ecuZoQCLcBGAs/s1600/4104_f6-1.PNG)
<b>Figure 6.1. Without the Option type, composing functions wouldn’t produce a function because the resulting program would potentially throw an Exception.</b>

### <font color='darkgreen'>Getting a value from an Option</font>
<b><font size='3ptx'>The first thing you’ll need is a way to retrieve the value in an <font color='blue'>Option</font></font></b>. One frequent use case when data is missing is to use a default value. The <font color='blue'>getOrElse</font> method that will return either the contained value if it exists:

In [29]:
import sys
import fpu
from fpu.fp import *

print("{}".format(sys.version_info))
print("Retrieve default: {}".format(Option.none().getOr('default value')))
print("Retrieve value: {}".format(Option.some('some value').getOr('default value')))

def getDefault():
    print('api:getDefault is called')
    return 0

# What if we only want api:getDefault to be called only when needed?
print("Retrieve: {}".format(Option.some(123).getOr(getDefault())))
print("Retrieve: {}".format(Option.none().getOr(getDefault())))

sys.version_info(major=3, minor=11, micro=2, releaselevel='final', serial=0)
Retrieve default: default value
Retrieve value: some value
api:getDefault is called
Retrieve: 123
api:getDefault is called
Retrieve: 0


Or you can use lazy evaluation for the <font color='blue'>getOrElse</font> method parameter:

In [7]:
print("Retrieve: {}".format(Option.some(123).getOrElse(getDefault)))
print("Retrieve: {}".format(Option.none().getOrElse(getDefault)))

Retrieve: 123
api:getDefault is called
Retrieve: 0


### <font color='darkgreen'>Applying functions to optional values</font>
One very important method in [**List**](https://github.com/johnklee/fpu/blob/master/fpu/flist.py) is the map method, which allows you to apply a function from `A` to `B` to each element of a list of `A`, producing a list of `B`. Considering that an <font color='blue'><b>Option</b></font> is like a list containing at most one element, you can apply the same principle.

In [9]:
some = Option.some(1)
none = Option.none()

double = lambda e: e * 2

print("Double {}={}".format(some, some.map(double)))
print("Double {}={}".format(none, none.map(double)))

Double Some(1)=Some(2)
Double None()=None()


### <font color='darkgreen'>Dealing with Option composition</font>
<b><font size='3ptx'>As you’ll soon realize, functions from `A` to `B` aren’t the most common ones in functional programming</font></b>. At first you may have trouble getting acquainted with functions returning optional values. After all, it seems to involve extra work to wrap values in <b><font color='blue'>Some</font></b> instances and later retrieve these values. But with further practice, you’ll see that these operations occur only rarely. When chaining functions to build a complex computation, you’ll often start with a value that’s returned by some previous computation and pass the result to a new function without seeing the intermediate result. In other words, you’ll more often use functions from `A` to <b><font color='blue'>Option\<B></font></b> than functions from `A` to `B`.
    
<font color='blue'>flatMap</font> instance method that takes as an argument a function from `A` to <b><font color='blue'>Option\<B></font></b> and returns an <b><font color='blue'>Option\<B></font></b>:

In [10]:
some = Option.some(1)
none = Option.none()
addOne = lambda e: Option.some(e + 1)
double = lambda e: Option.some(e * 2)

print("some.flatMap(addOne).flatMap(double)={}".format(some.flatMap(addOne).flatMap(double)))
print("none.flatMap(addOne).flatMap(double)={}".format(none.flatMap(addOne).flatMap(double)))

some.flatMap(addOne).flatMap(double)=Some(4)
none.flatMap(addOne).flatMap(double)=None()


Most of the case, there’s no need to “get” the value. Instead, you need to get the default value wrapped in <b><font color='blue'>Option</font></b> for chaining functions. Then <font color='blue'>orElse</font> will meet your need. For example:

In [12]:
some = Option.some(1)
none = Option.none()
def df():
    print('\tapi:df is called')
    return Option.some(-1)

print("some.OrElse(df)={}".format(some.orElse(df)))
print("none.OrElse(df)={}".format(none.orElse(df)))

some.OrElse(df)=Some(1)
	api:df is called
none.OrElse(df)=Some(-1)


More useful, you can use <font color='blue'>filter</font> to pass in the filter function which will return the original <font color='blue'><b>Some</b></font> object if True; otherwise, <font color='blue'><b>N</b></font> is returned:

In [13]:
some1 = Option.some(1)
some2 = Option.some(2)
none = Option.none()
bt1 = lambda e: e > 1  # Bigger than one

print("some1.filter(bt1)={}".format(some1.filter(bt1)))
print("some2.filter(bt1)={}".format(some2.filter(bt1)))
print("none.filter(bt1)={}".format(none.filter(bt1)))

some1.filter(bt1)=None()
some2.filter(bt1)=Some(2)
none.filter(bt1)=None()


### <font color='darkgreen'>Option use cases</font>
<b><font size='3ptx'>The best way to use <font color='blue'>Option</font> is through composition</font></b>. To do this, you must create all the necessary methods for all use cases. These use cases correspond to what you’d do with the value after testing that it’s not null. You could do one of the following:
* Use the value as the input to another function
* Apply an effect to the value
* Use the value if it’s not null, or use a default value to apply a function or an effect


Below is a classis example in using <font color='blue'><b>Option</b></font>:

In [109]:
DEFAULT_EAMIL = 'no email'
        
class User:
    def __init__(self, fn, sn, email=Option.none()):
        self.fn = fn
        self.sn = sn
        self.email = email
        
    def __str__(self):
        return "{}/{} ({})".format(self.fn, self.sn, self.email.getOr("No data"))

In [110]:
from fpu import flist

users = [
    Option.some(User("Mickey", "Mouse", Option.some("mickey@disney.com"))),
    Option.none(),
    Option.some(User("Minnie", "Mouse")),
]

In [111]:
for u in users:
    print(u.getOr('None'))

Mickey/Mouse (mickey@disney.com)
None
Minnie/Mouse (No data)


In [112]:
def get_email(u):
    return u.email

In [113]:
[u.flatMap(
    get_email,
).getOrElse(
    lambda: DEFAULT_EAMIL,
) for u in users]

['mickey@disney.com', 'no email', 'no email']

## <font color='darkblue'>Miscellaneous utilities for Option</font>
<font size='3ptx'><b>In order to make <font color='blue'>Option</font> as useful as possible, you need to add some utility methods</b></font>. Some of these methods are a must, and others are questionable because their use is not in the spirit of functional programming. You nevertheless must consider adding them. You may need a method to test whether an <font color='blue'><b>Option</b></font> is a <font color='blue'><b>N</b></font> or a <font color='blue'><b>Some</b></font>. You may also need an equals method for comparing options, in which case you mustn’t forget to define a <font color='blue'>compatible</font>, <font color='blue'>hashCode</font> method.

### <font color='darkgreen'>Testing for Some or None</font>
Until now, you haven’t needed to test an <font color='blue'><b>Option</b></font> to know whether it was a <font color='blue'><b>Some</b></font> or a <font color='blue'><b>N</b></font>. Ideally, you should never have to do this. In practice, though, there are times when it’s simpler to use this trick than to resort to real functional techniques. For example, you defined the <font color='blue'>map2</font> method as:

In [114]:
def map2(opa, opb, f):
    return opa.flatMap(lambda ax: opb.map(lambda bx: f(ax, bx)))

opa = Option.some(1)
opb = Option.some(2)
map2(opa, opb, lambda a, b: a + b).getOrElse(lambda e: 0)

3

This is very smart, and because you want to look smart, you might prefer this solution. But some may find the following version simpler to understand:

In [115]:
def map2_v2(opa, opb, f):
    return Option.some(f(opa.getOrThrow(), opb.getOrThrow())) if opa.isSome and opb.isSome else opOption.none()

map2_v2(Option.some(1), Option.some(2), lambda a, b: a + b).getOrThrow()

3

If you want to test this code, you’ll have to define the <font color='blue'>isSome</font> method first, <b>but this is not to encourage you to use this non-functional technique. You should always prefer the first form</b>, but you should also understand fully the relation between the two forms. Besides, you’ll probably find yourself needing the <font color='blue'>isSome</font> method someday.

### <font color='darkgreen'>equals and hashcode</font>